In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping


    
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        print(v)
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)
    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

def auc_roc_ignorante(y_true, y_pred):
    # any tensorflow metric
    value, _ = tf.metrics.auc(y_true,y_pred)
    return value

Using TensorFlow backend.


In [32]:
# generation a small dataset
N_all = 1000
N_tr = int(0.7 * N_all)
N_te = N_all - N_tr

np.random.seed(0)
x_dim = 50
y_dim = 30
X = np.random.rand(N_all,x_dim)
y = np.random.randint(0,2,size=(N_all-1,y_dim))
y_app = np.zeros(shape= (1,y_dim))
y_app[0] = 1
y = np.vstack((y,y_app))

X_train, X_valid = X[:N_tr, :], X[N_tr:, :]
y_train, y_valid = y[:N_tr, :], y[N_tr:, :]

In [37]:
# model & train
model = Sequential()
model.add(Dense(y_dim,input_shape=(X.shape[1],)))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', auc_roc])

my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

<tf.Variable 'metrics/auc_roc/auc/true_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics/auc_roc/auc/false_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics/auc_roc/auc/true_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics/auc_roc/auc/false_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/true_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/false_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/true_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/false_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_2/auc_roc/auc/true_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_2/auc_roc/auc/false_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_2/auc_roc/auc/true_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_2/auc_roc/auc/false_positives:0' sh

In [38]:
model.fit(X, y,batch_size=7000, epochs=20, verbose=1,callbacks=my_callbacks)

# # or use independent valid set
# model.fit(X_train, y_train,
#           validation_data=(X_valid, y_valid),
#           batch_size=32, nb_epoch=5, verbose=1,
#           callbacks=my_callbacks)

Epoch 1/20
1000/1000 [==============================] - 0s 336us/step - loss: 0.7296 - acc: 0.5037 - auc_roc: 0.0000e+00
Epoch 2/20
1000/1000 [==============================] - 0s 48us/step - loss: 0.7279 - acc: 0.5037 - auc_roc: 0.5045
Epoch 3/20
1000/1000 [==============================] - 0s 50us/step - loss: 0.7257 - acc: 0.5026 - auc_roc: 0.5046
Epoch 4/20
1000/1000 [==============================] - 0s 45us/step - loss: 0.7238 - acc: 0.5029 - auc_roc: 0.5047
Epoch 5/20
1000/1000 [==============================] - 0s 46us/step - loss: 0.7219 - acc: 0.5044 - auc_roc: 0.5048
Epoch 6/20
1000/1000 [==============================] - 0s 45us/step - loss: 0.7202 - acc: 0.5041 - auc_roc: 0.5049
Epoch 7/20
1000/1000 [==============================] - 0s 46us/step - loss: 0.7186 - acc: 0.5048 - auc_roc: 0.5050
Epoch 8/20
1000/1000 [==============================] - 0s 50us/step - loss: 0.7170 - acc: 0.5052 - auc_roc: 0.5050
Epoch 9/20
1000/1000 [==============================] - 0s 52us/ste

In [39]:
y_pred = model.predict(X_train)

In [40]:
aucs = []
for i in range(y.shape[1]):
    fpr, tpr, thresholds = roc_curve(y_train[:,i], y_pred[:,i],)
    aucs.append(auc(fpr,tpr))
    print(aucs[i])
np.mean(aucs)

0.4891169458868097
0.5054022687894341
0.5253284987464374
0.5133837827247569
0.5044308613790903
0.5112280356674535
0.5160102544005748
0.5131949908208759
0.5229251045205121
0.5420289145217512
0.4729966296914368
0.5305150901489417
0.4727066392345122
0.473072055521535
0.5187552551449399
0.4649505241500931
0.4882142507105752
0.4844264236306768
0.4991669116926394
0.516975056392821
0.5255245414103016
0.5097102833492518
0.5244956772334294
0.503070992191839
0.5011519607843138
0.5462278861304095
0.4803769328843702
0.5399428104575164
0.5475259406813562
0.46078335133012666


0.5067879623409594

In [41]:
roc_auc_score(y_train, y_pred)

0.5067879623409594